In [16]:
# @title Import Packages

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import interactive
import re
from functools import partial
%matplotlib inline

In [14]:
# @title Class Definition

# GcodeFileHandler

## Display "Upload GCODE" button
## When "Upload GCODE" is pressed; stores uploaded gcode files in dictionary gcode_storage --> {key: Filename, value: file contents}

class GCodeFileHandler:
    def __init__(self):
        self.file_select = widgets.FileUpload(
            accept='.gcode',
            multiple=True,
            description='Select GCODE'
        )
        self.upload_button = widgets.Button(description="Upload File(s)")
        self.reset_button = widgets.Button(description="Reset")
        self.gcode_storage = {}

        self.upload_button.on_click(self.on_upload_button_clicked)
        self.reset_button.on_click(self.on_reset_button_clicked)

        display(self.file_select)
        display(self.upload_button)
        display(self.reset_button)

    def on_upload_button_clicked(self, b):
        uploaded_files = self.file_select.value
        if uploaded_files:
            for filename, file_info in uploaded_files.items():
                self.gcode_storage[filename] = file_info['content'].decode('utf-8')
            print(f"Uploaded {len(uploaded_files)} file(s).")
            for filename in self.gcode_storage:
                print(f"Stored: {filename}")
        else:
            print("No files uploaded.")

    def on_reset_button_clicked(self, b):
        self.gcode_storage.clear()

        self.file_select.value.clear()

        clear_output(wait=True)

        display(self.file_select)
        display(self.upload_button)
        display(self.reset_button)
        print("Reset successful.")

# GcodeProcessor

## Display "Calculate Cost" button in corresponding cell
## When "Calculate Cost" is pressed; use dictionary to create an instance of GcodeProcessor for each key-value pair in gcode_storage; each instance takes the key and value of one element as input,
### storing the output in dictionary, filename_factors, --> {key: name of data, value: data}

class GcodeProcessor:
    def __init__(self, gcode_content):
        self.gcode = gcode_content

    def extract_layers(self):
        num_layers = self.gcode.count(';LAYER_CHANGE')

        return num_layers

    def extract_print_time(self):
        # Regular expression to match the estimated print time from gcode
        pattern = r"estimated printing time \(normal mode\) = (\d+h \d+m \d+s)"
        match = re.search(pattern, self.gcode)

        if match:
            print_time_str = match.group(1)
            total_hours = self.convert_print_time_to_hours(print_time_str)

            return total_hours, print_time_str
        else:
            return "Print time not found"

    def convert_print_time_to_hours(self, print_time_str):
        # Extract hours, minutes, and seconds from the print time string
        pattern = r"(?:(\d+)h )?(?:(\d+)m )?(?:(\d+)s)?"
        match = re.search(pattern, print_time_str)

        if match:
            hours = int(match.group(1) or 0)
            minutes = int(match.group(2) or 0)
            seconds = int(match.group(3) or 0)
            total_seconds = hours * 3600 + minutes * 60 + seconds
            total_hours = total_seconds / 3600
            return total_hours
        else:
            return 0  # Default to 0 if the pattern doesn't match

    def extract_filament_type(self):
        # Regular expression to match the filament type
        pattern = r"filament_type = (\w+)"
        match = re.search(pattern, self.gcode)

        if match:
            return match.group(1)
        else:
            return "Filament type not found"

    def extract_filament_usage_g(self):
        # Regular expression to match the filament usage in grams
        pattern = r"; filament used \[g\] = ([\d.]+)"
        match = re.search(pattern, self.gcode)

        if match:
            return float(match.group(1))
        else:
            return "Filament usage (g) not found"

    def extract_filament_usage_cm3(self):
        # Regular expression to match the filament usage in cubic centimeters
        pattern = r"; filament used \[cm3\] = ([\d.]+)"
        match = re.search(pattern, self.gcode)

        if match:
            return float(match.group(1))
        else:
            return "Filament usage (cm3) not found"

# CostCalculator

## Takes the data pulled from the Gcode in GcodeProcessor as input and returns the dataframe, cost_df, containing the types of cost and their values
## Displays cost dataframe


class CostCalculator:
    def __init__(self, print_hrs, fil_used_cm3):
        # define default values for cost calculation
        self.fil_cost = 0.05
        self.labor_rate = 20
        self.kwh_cost = 0.203
        self.machine_cost = 3000

        # initiate input params
        self.print_hrs = print_hrs
        self.fil_used_cm3 = fil_used_cm3

    def costFunc(self):

        # Calculate costs
        maint_cost_per_hr = (self.machine_cost / 43800) # 5 years in hours
        # total material cost
        filament_cost = self.fil_used_cm3 * self.fil_cost
        # toal labor cost
        labor_cost = (self.print_hrs/20) * self.labor_rate
        # total electricity cost
        elect_cost = self.kwh_cost * self.print_hrs
        # total depreciation from machine cost
        maint_this_print = maint_cost_per_hr * self.print_hrs

        total_cost = filament_cost + labor_cost + elect_cost + maint_this_print

        # Create DataFrame for cost breakdown
        data = {
            'Factor': ['Material', 'Labor', 'Electricty', 'Maintainence', 'Total'],
            'Cost ($)': [filament_cost, labor_cost, elect_cost, maint_this_print, total_cost]
        }
        cost_df = pd.DataFrame(data)
        return cost_df


# GcodeFileSelecter

## Creates a dropdown menu from which the user can select a gcodefile to explore cost factors with
class GCodeFileSelector:
    def __init__(self, gcode_storage):
        self.gcode_storage = gcode_storage # dict
        self.selected_file = None # returned from self.dropdown.value str
        self.print_hrs = None # float
        self.fil_used_cm3 = None # float
        self.menu = self.gcode_storage.keys() # all keys from input dict
        self.gcode_selected = None

        self.dropdown = widgets.Dropdown(
            options=self.menu,
            # options=self.gcode_storage.keys(),
            value=None,
            description='GCODE Files:'
        )

        self.confirm_button = widgets.Button(description='Confirm Selection')

        self.confirm_button.on_click(self.on_confirm_button_clicked)

        display(self.dropdown)
        display(self.confirm_button)



    def on_confirm_button_clicked(self, b):
        if self.dropdown.value:
            self.selected_file = self.dropdown.value
            print(f"Confirmed file: {self.selected_file}")


            self.gcode_selected = self.gcode_storage[self.selected_file]
            self.process_gcode_instance = GcodeProcessor(self.gcode_selected)

            self.print_hrs, self.print_str = self.process_gcode_instance.extract_print_time()
            self.fil_used_cm3 = self.process_gcode_instance.extract_filament_usage_cm3()


        else:
            print("No file selected.")

    def get_selected_file(self):
        return self.selected_file, self.print_hrs, self.fil_used_cm3


# CostExplorer
## Prompt user to select file from gcode_storage from dropdown menu
## Display sliders for certain cost parameters, with default settings matching those used in CostCalculator
## Display live pie chart with cost factors as the slices for visualization

class CostExploreWidgets:
    def __init__(self, print_hrs, fil_used_cm3):
        self.print_hrs = print_hrs
        self.fil_used_cm3 = fil_used_cm3
        self.p = self.print_hrs
        self.f = self.fil_used_cm3

    def gen_widgets(self):
        self.fil_cost_slider = widgets.FloatSlider(value=0.05, min=0, max=1, step=0.01, description='Filament Cost per cm3:', layout=widgets.Layout(width='600px'), style={'description_width': '150px'})
        self.labor_rate_slider = widgets.FloatSlider(value=20, min=0, max=50, step=1, description='Labor Cost per Hour:', layout=widgets.Layout(width='600px'), style={'description_width': '150px'})
        self.kwh_cost_slider = widgets.FloatSlider(value=0.203, min=0, max=1, step=0.001, description='Cost per kWh:', layout=widgets.Layout(width='600px'), style={'description_width': '150px'})
        self.machine_cost_slider = widgets.FloatSlider(value=3000, min=0, max=10000, step=100, description='Cost of 3D Printer:', layout=widgets.Layout(width='600px'), style={'description_width': '150px'})
        self.print_hrs_slider = widgets.FloatSlider(value=self.p, min=0, max=(self.p*10), step=1, description='Time to Print in Hours:', layout=widgets.Layout(width='600px'), style={'description_width': '150px'})
        self.fil_used_slider = widgets.FloatSlider(value=self.f, min=0, max=(self.f*10), step=1, description='Filament Used in cm3:', layout=widgets.Layout(width='600px'), style={'description_width': '150px'})

        output = widgets.Output()
        with output:
            display(self.fil_cost_slider, self.labor_rate_slider, self.kwh_cost_slider, self.machine_cost_slider, self.print_hrs_slider, self.fil_used_slider)
        display(output)

    def calc_slices(self):
        fil_cost = self.fil_cost_slider.value
        labor_rate = self.labor_rate_slider.value
        kwh_cost = self.kwh_cost_slider.value
        machine_cost = self.machine_cost_slider.value
        print_hrs = self.print_hrs_slider.value
        fil_used = self.fil_used_slider.value

        maint_cost_per_hr = (machine_cost / 43800) # 5 years in hours
        material_cost = fil_used * fil_cost
        labor_cost = (print_hrs / 20) * labor_rate
        electricity_cost = kwh_cost * print_hrs
        maintenance_cost = maint_cost_per_hr * print_hrs

        self.slices = {
            'Material': material_cost,
            'Labor': labor_cost,
            'Electricity': electricity_cost,
            'Maintenance': maintenance_cost
        }
        self.total_cost = sum(self.slices.values())

    def plot_chart(self):
        self.calc_slices()
        outputChart = widgets.Output()
        with outputChart:
            clear_output(wait=True)

            self.fig, self.ax = plt.subplots()

            labels = list(self.slices.keys())
            sizes = list(self.slices.values())
            explode = (0.1, 0, 0, 0)

            self.pie_chart = self.ax.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%', shadow=False, startangle=140)
            self.ax.axis('equal')

            self.ax.set_title('Cost Factors')

            total_str = f'Total: ${self.total_cost:.2f}'
            self.ax.text(0, -1.2, total_str, fontsize=12, ha='center')

            self.fig.canvas.draw()
        display(outputChart)

# Controller

class CostController:
    def __init__(self, print_hrs, fil_used_cm3):
        self.explore = CostExploreWidgets(print_hrs, fil_used_cm3)

    def run(self):
        self.explore.gen_widgets()

        gen_button = widgets.Button(description="Generate Chart", layout=widgets.Layout(width='800px'))
        reset_button = widgets.Button(description="Restore Defaults", layout=widgets.Layout(width='800px'))

        def gen_chart(b):
            self.explore.plot_chart()

        def reset_sliders(b):
            clear_output(wait=True)
            display(gen_button)
            display(reset_button)
            self.explore.gen_widgets()
            self.explore.plot_chart()

        gen_button.on_click(gen_chart)
        display(gen_button)

        reset_button.on_click(reset_sliders)
        display(reset_button)




In [15]:
# @title Upload Gcode File(s)

# Instance of GcodeFileHandler

gcode_handler = GCodeFileHandler()

FileUpload(value={}, accept='.gcode', description='Select GCODE', multiple=True)

Button(description='Upload File(s)', style=ButtonStyle())

Button(description='Reset', style=ButtonStyle())

In [9]:
# @title Gcode Details

get_details_b = widgets.Button(description='Get Print Details')
display(get_details_b)

def process_instances(b):
    for filename, content in gcode_handler.gcode_storage.items():
        processor = GcodeProcessor(content)

        num_layers = processor.extract_layers()
        print_hrs, print_time = processor.extract_print_time()
        fil_type = processor.extract_filament_type()
        fil_used_g = processor.extract_filament_usage_g()
        fil_used_cm3 = processor.extract_filament_usage_cm3()

        print(f"Print Details for {filename}:")
        print(f"Number of layers: {num_layers}")
        print(f"Print Time (hrs): {print_time}")
        print(f"Filament Type: {fil_type}")
        print(f"Filament Used (g): {fil_used_g}")
        print(f"Filamaent Used (cm3): {fil_used_cm3}")
        print(" ")

get_details_b.on_click(process_instances)

Button(description='Get Print Details', style=ButtonStyle())

In [8]:
# @title Calculate Costs

# Instance of CostCalculator
get_cost = widgets.Button(description='Calculate Costs')
display(get_cost)

cost_compare = {}

def cost_instances(b):
    global cost_compare
    for filename, content in gcode_handler.gcode_storage.items():
        processor = GcodeProcessor(content)

        print_hrs, print_time = processor.extract_print_time()
        fil_used_cm3 = processor.extract_filament_usage_cm3()

        cost_calc = CostCalculator(print_hrs, fil_used_cm3)
        cost_df = cost_calc.costFunc()

        print(f'Filename: {filename}')
        display(cost_df)

        cost_compare[filename] = cost_df
        print(" ")




get_cost.on_click(cost_instances)


# Instance of FactorVisualizer

Button(description='Calculate Costs', style=ButtonStyle())

In [7]:
# @title Select Gcode File to Explore Cost Factors
file_selector = GCodeFileSelector(gcode_handler.gcode_storage)

Dropdown(description='GCODE Files:', options=(), value=None)

Button(description='Confirm Selection', style=ButtonStyle())

In [6]:
# @title Activate the Cost Explorer

selected_file, print_hrs, fil_used_cm3 = file_selector.get_selected_file()
controls = CostController(print_hrs, fil_used_cm3)
controls.run()

NameError: name 'file_selector' is not defined